In [1]:
import sys
sys.path.append("..")
from utils import load_data
import torch
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torchvision import datasets, transforms
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
seeds = [1,2,3,4,5,6,7,8,9,10]

In [13]:
data_names = ["wisconsin", "cora", "citeseer", "pubmed", 
              "cornell", "texas", "chameleon", "squirrel", 
              "film", "photo", "computer"]

In [3]:
def data_loaders(X, y, train_idx, val_idx, test_idx, batch_size=32):
    train = torch.utils.data.TensorDataset(X[train_idx], y[train_idx])
    val = torch.utils.data.TensorDataset(X[val_idx], y[val_idx])
    test = torch.utils.data.TensorDataset(X[test_idx], y[test_idx])
    trainset = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    valset = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=False)
    testset = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    return trainset, valset, testset

In [4]:
class MLP(Module):
    def __init__(self, idim, nclass):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(idim, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, nclass)
        self.bn = nn.BatchNorm1d(100)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.bn(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
class SMLP(Module):
    def __init__(self, idim, nclass):
        super(SMLP, self).__init__()
        self.fc1 = nn.Linear(idim, 100)
        self.fc2 = nn.Linear(100, nclass)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
def evaluate_mlp(model_class, dname, split="0.6_0.2_0.2", bs=16, s=None):
    gnx, normed_adj, X, y, idx_train, idx_val, idx_test = load_data(dname, [''], split=split, random_state=s)
    trainloader, valloader, testloader = data_loaders(X, y, idx_train, idx_val, idx_test)
    batch_size = bs
    model = MLP(int(X.size(1)), int(y.max()+1))
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    obj = nn.CrossEntropyLoss()
    epochs = 500
    best_val = 0.0
    stats = {
        "tr_loss": [],
        "tr_acc": [],
        "tr_loss_e": [],
        "tr_acc_e": [],
        "va_loss": [],
        "va_acc": [],
        "va_loss_e": [],
        "va_acc_e": [],
        "te_acc": [],
        "te_loss": [],
        "best_epochs": []
    }
    # Train
    for epoch in range(1, epochs+1):
        model.train() 
        train_loss, train_acc = 0.0, 0.0
        ndata = len(trainloader.dataset)
        for i, (X,y) in enumerate(trainloader):
            optimizer.zero_grad()
            out = model(X)
            pred = out.data.max(1)[1]
            loss = obj(out, y)
            loss.backward()
            optimizer.step()
    
            stats["tr_loss"].append(loss.item())
            stats["tr_acc"].append(np.sum(pred.cpu().numpy() == y.data.cpu().numpy()) / len(y))
            train_loss += stats["tr_loss"][-1]
            train_acc += np.sum(pred.cpu().numpy() == y.data.cpu().numpy())
            
            # Eval per batch
            model.eval() 
            val_loss, val_acc = 0.0, 0.0
            ndata = len(valloader.dataset)
            for i, (X,y) in enumerate(valloader):
                out = model(X)
                pred = out.data.max(1)[1]
                loss = obj(out, y)
                val_loss += loss.item()
                val_acc += np.sum(pred.cpu().numpy() == y.data.cpu().numpy())
        
            stats["va_loss"].append(val_loss / ndata)
            stats["va_acc"].append(val_acc / ndata * 100)
    
        stats["tr_loss_e"].append(train_loss / ndata)
        stats["tr_acc_e"].append(train_acc / ndata * 100)
    
        # Eval per epoch
        model.eval() 
        val_loss, val_acc = 0.0, 0.0
        ndata = len(valloader.dataset)
        for i, (X,y) in enumerate(valloader):
            out = model(X)
            pred = out.data.max(1)[1]
            loss = obj(out, y)
            val_loss += loss.item()
            val_acc += np.sum(pred.cpu().numpy() == y.data.cpu().numpy())

        stats["va_loss_e"].append(val_loss / ndata)
        stats["va_acc_e"].append(val_acc / ndata * 100)
        
        # Check if best val
        if (stats["va_acc_e"][-1] > best_val):
            best_val = stats["va_acc_e"][-1]
            # Test
            model.eval() 
            te_loss, te_acc = 0.0, 0.0
            ndata = len(testloader.dataset)
            for i, (X,y) in enumerate(testloader):
                out = model(X)
                pred = out.data.max(1)[1]
                loss = obj(out, y)
                te_loss += loss.item()
                te_acc += np.sum(pred.cpu().numpy() == y.data.cpu().numpy())
            
            stats['te_loss'].append(te_loss / ndata)
            stats['te_acc'].append(te_acc / ndata * 100)
            stats['best_epochs'].append(epoch)
            
    print(stats['te_acc'][-1])
    return stats

In [9]:
stat_wis = [evaluate_mlp(MLP, "wisconsin", s=seeds[i]) for i in range(10)]

86.27450980392157
88.23529411764706
88.23529411764706
88.23529411764706
80.3921568627451
86.27450980392157
80.3921568627451
88.23529411764706
82.35294117647058
84.31372549019608


In [10]:
np.sum([stat_wis[i]['te_acc'][-1] for i in range(10)]) / 10

85.29411764705883

In [11]:
np.std([stat_wis[i]['te_acc'][-1] for i in range(10)]) / 10

0.306911291029383

In [12]:
stat_wis_smlp = [evaluate_mlp(SMLP, "wisconsin", s=seeds[i]) for i in range(10)]

80.3921568627451
82.35294117647058
86.27450980392157
90.19607843137256
80.3921568627451
80.3921568627451
82.35294117647058
88.23529411764706
84.31372549019608
86.27450980392157


In [110]:
np.sum([stat_wis_smlp[i]['te_acc'][-1] for i in range(10)]) / 10

83.13725490196079

In [111]:
np.std([stat_wis_smlp[i]['te_acc'][-1] for i in range(10)]) / 10

0.42236586722623565

In [ ]:
all_stats = dict({
    "wisconsin": stat_wis
})
for dn in data_names[1:]:
    all_stats[dn] = [evaluate_mlp(MLP, dn, s=seeds[i]) for i in range(10)]
    print(dn)
    print(np.average([all_stats[dn][i]['te_acc'][-1] for i in range(10)]))
    print(np.std([all_stats[dn][i]['te_acc'][-1] for i in range(10)]))

70.84870848708486
70.47970479704797
68.26568265682657
69.74169741697416
68.26568265682657
68.81918819188192
